# Comparing Reductions


For your first task, the code is already written for you. We will compare 3 of the reductions given during the presentation: the naive atomic-only reduction, the classical parallel reduction with atomic finish, and the warp shuffle reduction (with atomic finish).

In [1]:
%%writefile reductions.cu

#include <stdio.h>

// error checking macro
#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)


const size_t N = 8ULL*1024ULL*1024ULL;  // data size
//const size_t N = 256*640; // data size
const int BLOCK_SIZE = 256;  // CUDA maximum is 1024
// naive atomic reduction kernel
__global__ void atomic_red(const float *gdata, float *out){
  size_t idx = threadIdx.x+blockDim.x*blockIdx.x;
  if (idx < N) atomicAdd(out, gdata[idx]);
}

__global__ void reduce(float *gdata, float *out){
     __shared__ float sdata[BLOCK_SIZE];
     int tid = threadIdx.x;
     sdata[tid] = 0.0f;
     size_t idx = threadIdx.x+blockDim.x*blockIdx.x;

     while (idx < N) {  // grid stride loop to load data
        sdata[tid] += gdata[idx];
        idx += gridDim.x*blockDim.x;
        }

     for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
        __syncthreads();
        if (tid < s)  // parallel sweep reduction
            sdata[tid] += sdata[tid + s];
        }
     if (tid == 0) out[blockIdx.x] = sdata[0];
  }

 __global__ void reduce_a(float *gdata, float *out){
     __shared__ float sdata[BLOCK_SIZE];
     int tid = threadIdx.x;
     sdata[tid] = 0.0f;
     size_t idx = threadIdx.x+blockDim.x*blockIdx.x;

     while (idx < N) {  // grid stride loop to load data
        sdata[tid] += gdata[idx];
        idx += gridDim.x*blockDim.x;
        }

     for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
        __syncthreads();
        if (tid < s)  // parallel sweep reduction
            sdata[tid] += sdata[tid + s];
        }
     if (tid == 0) atomicAdd(out, sdata[0]);
  }


__global__ void reduce_ws(float *gdata, float *out){
     __shared__ float sdata[32];
     int tid = threadIdx.x;
     int idx = threadIdx.x+blockDim.x*blockIdx.x;
     float val = 0.0f;
     unsigned mask = 0xFFFFFFFFU;
     int lane = threadIdx.x % warpSize;
     int warpID = threadIdx.x / warpSize;
     while (idx < N) {  // grid stride loop to load
        val += gdata[idx];
        idx += gridDim.x*blockDim.x;
        }

 // 1st warp-shuffle reduction
    for (int offset = warpSize/2; offset > 0; offset >>= 1)
       val += __shfl_down_sync(mask, val, offset);
    if (lane == 0) sdata[warpID] = val;
   __syncthreads(); // put warp results in shared mem

// hereafter, just warp 0
    if (warpID == 0){
 // reload val from shared mem if warp existed
       val = (tid < blockDim.x/warpSize)?sdata[lane]:0;

 // final warp-shuffle reduction
       for (int offset = warpSize/2; offset > 0; offset >>= 1)
          val += __shfl_down_sync(mask, val, offset);

       if  (tid == 0) atomicAdd(out, val);
     }
  }




int main(){

  float *h_A, *h_sum, *d_A, *d_sum;
  h_A = new float[N];  // allocate space for data in host memory
  h_sum = new float;
  for (int i = 0; i < N; i++)  // initialize matrix in host memory
    h_A[i] = 1.0f;
  cudaMalloc(&d_A, N*sizeof(float));  // allocate device space for A
  cudaMalloc(&d_sum, sizeof(float));  // allocate device space for sum
  cudaCheckErrors("cudaMalloc failure"); // error checking
  // copy matrix A to device:
  cudaMemcpy(d_A, h_A, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaCheckErrors("cudaMemcpy H2D failure");
  cudaMemset(d_sum, 0, sizeof(float));
  cudaCheckErrors("cudaMemset failure");
  //cuda processing sequence step 1 is complete
  atomic_red<<<(N+BLOCK_SIZE-1)/BLOCK_SIZE, BLOCK_SIZE>>>(d_A, d_sum);
  cudaCheckErrors("atomic reduction kernel launch failure");
  //cuda processing sequence step 2 is complete
  // copy vector sums from device to host:
  cudaMemcpy(h_sum, d_sum, sizeof(float), cudaMemcpyDeviceToHost);
  //cuda processing sequence step 3 is complete
  cudaCheckErrors("atomic reduction kernel execution failure or cudaMemcpy H2D failure");
  if (*h_sum != (float)N) {printf("atomic sum reduction incorrect!\n"); return -1;}
  printf("atomic sum reduction correct!\n");
  const int blocks = 640;
  cudaMemset(d_sum, 0, sizeof(float));
  cudaCheckErrors("cudaMemset failure");
  //cuda processing sequence step 1 is complete
  reduce_a<<<blocks, BLOCK_SIZE>>>(d_A, d_sum);
  cudaCheckErrors("reduction w/atomic kernel launch failure");
  //cuda processing sequence step 2 is complete
  // copy vector sums from device to host:
  cudaMemcpy(h_sum, d_sum, sizeof(float), cudaMemcpyDeviceToHost);
  //cuda processing sequence step 3 is complete
  cudaCheckErrors("reduction w/atomic kernel execution failure or cudaMemcpy H2D failure");
  if (*h_sum != (float)N) {printf("reduction w/atomic sum incorrect!\n"); return -1;}
  printf("reduction w/atomic sum correct!\n");
  cudaMemset(d_sum, 0, sizeof(float));
  cudaCheckErrors("cudaMemset failure");
  //cuda processing sequence step 1 is complete
  reduce_ws<<<blocks, BLOCK_SIZE>>>(d_A, d_sum);
  cudaCheckErrors("reduction warp shuffle kernel launch failure");
  //cuda processing sequence step 2 is complete
  // copy vector sums from device to host:
  cudaMemcpy(h_sum, d_sum, sizeof(float), cudaMemcpyDeviceToHost);
  //cuda processing sequence step 3 is complete
  cudaCheckErrors("reduction warp shuffle kernel execution failure or cudaMemcpy H2D failure");
  if (*h_sum != (float)N) {printf("reduction warp shuffle sum incorrect!\n"); return -1;}
  printf("reduction warp shuffle sum correct!\n");
  return 0;
}

Writing reductions.cu


In [2]:
!nvcc -arch=sm_75 reductions.cu -o reductions
!./reductions

atomic sum reduction correct!
reduction w/atomic sum correct!
reduction warp shuffle sum correct!


##5.1 Max-finding reduction

For this exercise, you are given a fully-functional sum-reduction code, similar to the code used for exercise 1 above, except that we will use the 2-stage reduction method without atomic finish. If you wish you can compile and run it as-is to see how it works. Your task is to modify it (only the kernel) so that it creates a proper max-finding reduction. That means that the kernel should report the maximum value in the data set, rather than the sum of the data set. You are expected to use a similar parallel-sweep-reduction technique. If you need help, refer to the solution.

###max_reduction.cu

```
#include <stdio.h>

// error checking macro
#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)


const size_t N = 8ULL*1024ULL*1024ULL;  // data size
const int BLOCK_SIZE = 256;  // CUDA maximum is 1024

__global__ void reduce(float *gdata, float *out, size_t n){
     __shared__ float sdata[BLOCK_SIZE];
     int tid = threadIdx.x;
     sdata[tid] = 0.0f;
     size_t idx = threadIdx.x+blockDim.x*blockIdx.x;

     while (idx < n) {  // grid stride loop to load data
        sdata[tid] += gdata[idx];
        idx += gridDim.x*blockDim.x;  
        }

     for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
        __syncthreads();
        if (tid < s)  // parallel sweep reduction
            sdata[tid] += sdata[tid + s];
        }
     if (tid == 0) out[blockIdx.x] = sdata[0];
  }

int main(){

  float *h_A, *h_sum, *d_A, *d_sums;
  const int blocks = 640;
  h_A = new float[N];  // allocate space for data in host memory
  h_sum = new float;
  float max_val = 5.0f;
  for (size_t i = 0; i < N; i++)  // initialize matrix in host memory
    h_A[i] = 1.0f;
  h_A[100] = max_val;
  cudaMalloc(&d_A, N*sizeof(float));  // allocate device space for A
  cudaMalloc(&d_sums, blocks*sizeof(float));  // allocate device space for partial sums
  cudaCheckErrors("cudaMalloc failure"); // error checking
  // copy matrix A to device:
  cudaMemcpy(d_A, h_A, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaCheckErrors("cudaMemcpy H2D failure");
  //cuda processing sequence step 1 is complete
  reduce<<<blocks, BLOCK_SIZE>>>(d_A, d_sums, N); // reduce stage 1
  cudaCheckErrors("reduction kernel launch failure");
  reduce<<<1, BLOCK_SIZE>>>(d_sums, d_A, blocks); // reduce stage 2
  cudaCheckErrors("reduction kernel launch failure");
  //cuda processing sequence step 2 is complete
  // copy vector sums from device to host:
  cudaMemcpy(h_sum, d_A, sizeof(float), cudaMemcpyDeviceToHost);
  //cuda processing sequence step 3 is complete
  cudaCheckErrors("reduction w/atomic kernel execution failure or cudaMemcpy D2H failure");
  printf("reduction output: %f, expected sum reduction output: %f, expected max reduction output: %f\n", *h_sum, (float)((N-1)+max_val), max_val);
  return 0;
}
```

In [1]:
%%writefile max_reduction.cu
#include <stdio.h>

// error checking macro
#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)


const size_t N = 8ULL*1024ULL*1024ULL;  // data size
const int BLOCK_SIZE = 256;  // CUDA maximum is 1024

__global__ void reduce(float *gdata, float *out, size_t n){
     __shared__ float sdata[BLOCK_SIZE];
     int tid = threadIdx.x;
     sdata[tid] = 0.0f;
     size_t idx = threadIdx.x+blockDim.x*blockIdx.x;

     while (idx < n) {  // grid stride loop to load data
        sdata[tid] = max(sdata[tid],gdata[idx]);
        idx += gridDim.x*blockDim.x;
        }

     for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
        __syncthreads();
        if (tid < s)  // parallel sweep reduction
            sdata[tid] = max(sdata[tid], sdata[tid + s]);
        }
     if (tid == 0) out[blockIdx.x] = sdata[0];
  }

int main(){

  float *h_A, *h_sum, *d_A, *d_sums;
  const int blocks = 640;
  h_A = new float[N];  // allocate space for data in host memory
  h_sum = new float;
  float max_val = 5.0f;
  for (size_t i = 0; i < N; i++)  // initialize matrix in host memory
    h_A[i] = 1.0f;
  h_A[100] = max_val;
  cudaMalloc(&d_A, N*sizeof(float));  // allocate device space for A
  cudaMalloc(&d_sums, blocks*sizeof(float));  // allocate device space for partial sums
  cudaCheckErrors("cudaMalloc failure"); // error checking
  // copy matrix A to device:
  cudaMemcpy(d_A, h_A, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaCheckErrors("cudaMemcpy H2D failure");
  //cuda processing sequence step 1 is complete
  reduce<<<blocks, BLOCK_SIZE>>>(d_A, d_sums, N); // reduce stage 1
  cudaCheckErrors("reduction kernel launch failure");
  reduce<<<1, BLOCK_SIZE>>>(d_sums, d_A, blocks); // reduce stage 2
  cudaCheckErrors("reduction kernel launch failure");
  //cuda processing sequence step 2 is complete
  // copy vector sums from device to host:
  // We are copying only the first value of d_A, which is the max value. The rest of the values are not relevant.
  cudaMemcpy(h_sum, d_A, sizeof(float), cudaMemcpyDeviceToHost);
  //cuda processing sequence step 3 is complete
  cudaCheckErrors("reduction w/atomic kernel execution failure or cudaMemcpy D2H failure");
  printf("reduction output: %f, expected sum reduction output: %f, expected max reduction output: %f\n", *h_sum, (float)((N-1)+max_val), max_val);
  return 0;
}

Writing max_reduction.cu


In [2]:
!nvcc -arch=sm_75 max_reduction.cu -o max_reduction
!./max_reduction

reduction output: 5.000000, expected sum reduction output: 8388612.000000, expected max reduction output: 5.000000
